This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [45]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [289]:
# First, import the relevant modules
import requests
import collections

In [290]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']
url = "https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?start_date=2014-01-01&end_date=2014-01-05&frequency=daily&api_key="+API_KEY
r = requests.get(url)
json_data = r.json()

In [291]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
for key, value in json_data.items():
    print(key + ':', value)

dataset_data: {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'], 'start_date': '2014-01-01', 'end_date': '2014-01-05', 'frequency': 'daily', 'data': [['2014-01-03', 55.02, 55.65, 54.53, 54.557, 38246200.0, 0.0, 1.0, 55.02, 55.65, 54.53, 54.557, 38246200.0], ['2014-01-02', 54.83, 55.22, 54.19, 54.71, 43195500.0, 0.0, 1.0, 54.83, 55.22, 54.19, 54.71, 43195500.0]], 'collapse': None, 'order': None}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# data structure build out

In [292]:
# 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
#    absfor the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&frequency=daily&api_key="+API_KEY
r = requests.get(url)
# 2: Convert the returned JSON object into a Python dictionary.
json_data = r.json()

In [294]:
# examining structure and values of originating dictionary
for key, value in json_data.items():
    print(key + ':', value)

In [295]:
# create empty dictionary: my_dict
my_dict = {}
# pull value points out into list of lists
values = json_data['dataset_data']['data']

In [296]:
# add key+value pairs to my_dict; note that value points are not yet labeled
for ele, value in enumerate(values):
    my_dict[ele] = value

In [297]:
# create list of column names to label value points
column_names = [i for i in json_data['dataset_data']['column_names']]

In [298]:
column_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [299]:
# assign the zipped dictionary value of column names + values to my_dict to create a nested dictionary
for key in my_dict.keys():
    my_dict[key] = dict(zip(column_names,my_dict[key]))  

# questions 3 - 7

# 3 Calculate what the highest and lowest opening prices were for the stock in this period.

In [301]:
# Create list of opening prices
opening_prices = []
for key in my_dict.keys():
    opening_prices.append(my_dict[key]['Open'])

In [303]:
# Discover that some opening prices are null
price_types = []
for items in opening_prices:
    price_types.append(type(items))

In [304]:
highest_opening = max(x for x in opening_prices if x is not None)
lowest_opening = min(x for x in opening_prices if x is not None)
print('The highest opening price in 2017 was $' + str(highest_opening))
print('The lowest opening price in 2017 was $' + str(lowest_opening))

The highest opening price in 2017 was $53.11
The lowest opening price in 2017 was $34.0


# 4 What was the largest change in any one day (based on High and Low price)?

In [305]:
# Create a 'Change' dictionary key + value using the Range and add those ranges to a list
daily_changes = []
for key in my_dict.keys():
    my_dict[key]['Change'] = my_dict[key]['High'] - my_dict[key]['Low']
    daily_changes.append(my_dict[key]['Change'])

In [306]:
# Find the max value in the list
largest_change = max(x for x in daily_changes)
print('The largest daily change in 2017 was $' + str(round(largest_change, 2)))

The largest daily change in 2017 was $2.81


# 5 What was the largest change between any two days (based on Closing Price)?

In [307]:
change_between = []
# Create a list of key values
keys = list(my_dict.keys())
# Iterate over the key values starting with the second one. Compare each Close value to the previous day's close value and append to list
for key in keys[1:]:
    my_dict[key]['Change_from_Prev'] = my_dict[key]['Close'] - my_dict[key-1]['Close']
    change_between.append(my_dict[key]['Change_from_Prev'])
# Find max value in list
largest_change_from_previous = max(x for x in change_between)
print('The largest change between any two days in 2017 was $' + str(round(largest_change_from_previous, 2)))

The largest change between any two days in 2017 was $2.56


# 6 What was the average daily trading volume during this year?

In [308]:
daily_volumes = []
# Create list of Traded Volumes
for key in my_dict.keys():
    daily_volumes.append(my_dict[key]['Traded Volume'])


In [309]:
# Find average of Traded Volume list
average = sum(daily_volumes) / len(daily_volumes)
print('The average daily trading volume in 2017 was ' + str(int(average)))

The average daily trading volume in 2017 was 89124


# 7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [310]:
# Function to find median based on if length is even or odd
def find_median(lst):
    length = len(lst)
    lst.sort()
    # even length
    if length % 2 == 0:
        left_index = int(length/2 -1) # get item to left of middle
        right_index = int(length/2) # get item to right of middle
        median = (lst[left_index] + lst[right_index])/2 # average left and right item
        return median
    # odd  length
    else: 
        median = lst[length//2]
        return median

In [311]:
find_median(daily_volumes)

76286.0